## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
import os
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Determine the number of unique values in each column.
attrition_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[attrition_cat].nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [3]:
# Look at APPLICATION_TYPE value counts for binning
app_count= application_df["APPLICATION_TYPE"].value_counts()
app_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_count[app_count<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df["CLASSIFICATION"].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts().loc[lambda x: x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_count[classification_count<1000].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df)
application_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [9]:
# Split our preprocessed data into our features and target arrays
y = application_dummies["IS_SUCCESSFUL"].values
X = application_dummies.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn_m = tf.keras.models.Sequential()


# First hidden layer
nn_m.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim= number_input_features, activation= "relu" ))


# Second hidden layer
nn_m.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_m.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_m.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 8)                 352       
                                                                 
 dense_13 (Dense)            (None, 5)                 45        
                                                                 
 dense_14 (Dense)            (None, 1)                 6         
                                                                 
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn_m.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Create a checkpoint to save weights every 5 epochs
checkpoint_path = "."
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
   checkpoint_path, verbose=1, save_weights_only=True,
   # Save weights, every epoch.
   save_freq='epoch')

nn_m.save_weights(checkpoint_path.format(epoch=5))
#StackOverflowsite: https://stackoverflow.com/questions/61046870/how-to-save-weights-of-keras-model-for-each-epoch#:~:text=

In [32]:
# Train the model
fit_model = nn_m.fit(X_train_scaled, y_train,
         epochs = 50, callbacks = [cp_callback],
         validation_data = (X_test,y_test),
         verbose=0)


Epoch 1: saving model to .

Epoch 2: saving model to .

Epoch 3: saving model to .

Epoch 4: saving model to .

Epoch 5: saving model to .

Epoch 6: saving model to .

Epoch 7: saving model to .

Epoch 8: saving model to .

Epoch 9: saving model to .

Epoch 10: saving model to .

Epoch 11: saving model to .

Epoch 12: saving model to .

Epoch 13: saving model to .

Epoch 14: saving model to .

Epoch 15: saving model to .

Epoch 16: saving model to .

Epoch 17: saving model to .

Epoch 18: saving model to .

Epoch 19: saving model to .

Epoch 20: saving model to .

Epoch 21: saving model to .

Epoch 22: saving model to .

Epoch 23: saving model to .

Epoch 24: saving model to .

Epoch 25: saving model to .

Epoch 26: saving model to .

Epoch 27: saving model to .

Epoch 28: saving model to .

Epoch 29: saving model to .

Epoch 30: saving model to .

Epoch 31: saving model to .

Epoch 32: saving model to .

Epoch 33: saving model to .

Epoch 34: saving model to .

Epoch 35: saving model

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_m.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5527 - accuracy: 0.7320 - 477ms/epoch - 2ms/step
Loss: 0.5526745915412903, Accuracy: 0.7320116758346558


In [36]:
# Export our model to HDF5 file
nn_m.save('AlphabetSoupCharity.h5')

In [52]:
# Optimize the model 

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 7
hidden_nodes_layer2 = 4

nn_opt = tf.keras.models.Sequential()


# First hidden layer
nn_opt.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim= number_input_features, activation= "relu" ))


# Second hidden layer
nn_opt.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_opt.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_opt.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 7)                 308       
                                                                 
 dense_22 (Dense)            (None, 4)                 32        
                                                                 
 dense_23 (Dense)            (None, 1)                 5         
                                                                 
Total params: 345
Trainable params: 345
Non-trainable params: 0
_________________________________________________________________


In [53]:
# Compile the model
nn_opt.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [54]:
# Create a checkpoint to save weights every 5 epochs
checkpoint_path = "."
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
   checkpoint_path, verbose=1, save_weights_only=True,
   # Save weights, every epoch.
   save_freq='epoch')

nn_m.save_weights(checkpoint_path.format(epoch=5))
#StackOverflowsite: https://stackoverflow.com/questions/61046870/how-to-save-weights-of-keras-model-for-each-epoch#:~:text=

In [55]:
# Train the model
fit_model = nn_m.fit(X_train_scaled, y_train,
         epochs = 50, callbacks = [cp_callback],
         validation_data = (X_test,y_test),
         verbose=0)


Epoch 1: saving model to .

Epoch 2: saving model to .

Epoch 3: saving model to .

Epoch 4: saving model to .

Epoch 5: saving model to .

Epoch 6: saving model to .

Epoch 7: saving model to .

Epoch 8: saving model to .

Epoch 9: saving model to .

Epoch 10: saving model to .

Epoch 11: saving model to .

Epoch 12: saving model to .

Epoch 13: saving model to .

Epoch 14: saving model to .

Epoch 15: saving model to .

Epoch 16: saving model to .

Epoch 17: saving model to .

Epoch 18: saving model to .

Epoch 19: saving model to .

Epoch 20: saving model to .

Epoch 21: saving model to .

Epoch 22: saving model to .

Epoch 23: saving model to .

Epoch 24: saving model to .

Epoch 25: saving model to .

Epoch 26: saving model to .

Epoch 27: saving model to .

Epoch 28: saving model to .

Epoch 29: saving model to .

Epoch 30: saving model to .

Epoch 31: saving model to .

Epoch 32: saving model to .

Epoch 33: saving model to .

Epoch 34: saving model to .

Epoch 35: saving model

In [56]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_opt.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.7224 - accuracy: 0.5291 - 545ms/epoch - 2ms/step
Loss: 0.7223734259605408, Accuracy: 0.5290961861610413
